# Universal Custom Estimators

## Patterns for Adding Custom Functionality

What can go wrong and why is it important to get this right

In [ ]:
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
import numpy as np

X_digits, y_digits = datasets.load_digits(return_X_y=True)

# Define a pipeline to search for the best combination of PCA truncation
# and classifier regularization.
pca = PCA()
# set the tolerance to a large value to make the example faster
logistic = LogisticRegression(max_iter=10000, tol=0.1)
pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])
# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {
    'pca__n_components': [5, 15, 30, 45, 64],
    'logistic__C': np.logspace(-4, 4, 4),
}

search = GridSearchCV(pipe, param_grid, n_jobs=-1)

X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits, random_state=123)

search.fit(X_train, y_train)

best = search.best_estimator_

print(f"Training set score: {best.score(X_train, y_train)}")
print(f"Test set score: {best.score(X_test, y_test)}")

In [ ]:
def mutate(X):
    """Mutates X"""
    # ... do something ...
    return X

pca = PCA(n_components=search.best_params_['pca__n_components'])
logistic = LogisticRegression(
    max_iter=10000, tol=0.1, C=search.best_params_['logistic__C'])

X_train, X_test, y_train, y_test = train_test_split(
    X_digits, y_digits, random_state=123)

X_train = pca.fit_transform(X_train, y_train)
X_train = mutate(X_train)
logistic = logistic.fit(X_train, y_train)

X_test = pca.transform(X_test) # <- Don't call fit again!
X_test = mutate(X_test) # <-Don’t forget to call mutate on X_test!

print(f"Training set score: {logistic.score(X_train, y_train)}")
print(f"Test set score: {logistic.score(X_test, y_test)}")

In [1]:
OR

NameError: name 'OR' is not defined

In [ ]:
class PcaLogRegBlock:
    def __init__(self, filepath, n_components, C):
        self.filepath = filepath
        self.n_components = n_components
        self.C = C
    def load_data(self):
        self.df = pd.read_csv(self.filepath)
    def add_features(self, X):
        """Fetches data from alternate source and merges into X.
        """
        # ... mutate X ...
        return X
    def build_model(self):
        X, y = self.df[self.features], self.df[self.target]
        pca = PCA(n_components=self.n_components)
        logistic = LogisticRegression(max_iter=10000, tol=0.1, C=self.C)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, random_state=123)
        X_train = pca.fit_transform(X_train, y_train)
        X_train = self.add_features(X_train)
        return logistic.fit(X_train, y_train)
    def run(self):
        self.load_data()
        model = self.build_model()
        self.save_model(model)

In [ ]:
class PcaLogRegGridSearchBlock:
    def __init__(self, filepath):
        self.filepath = filepath
        self.df = pd.read_csv(self.filepath)
        self.best_params_ = {}
    def run(self):
        X, y = self.df[self.features], self.df[self.target]
        param_grid = {
            'n_components': [5, 15, 30, 45, 64],
        }
        search = GridSearchCV(PCA(), param_grid, n_jobs=-1)
        search.fit(X, y)
        self.best_params_.update(search.best_params_)
        param_grid = {
            'C': np.logspace(-4, 4, 4),
        }
        search = GridSearchCV(LogisticRegression(max_iter=10000, tol=0.1), param_grid, n_jobs=-1)
        search.fit(X, y)
        self.best_params_.update(search.best_params_)

In [ ]:
def driver():
    gs = PcaLogRegGridSearchBlock('/path/to/file', ...)
    gs.run()
    # Use best results from grid search to retrain
    pca_logreg = PcaLogRegBlock('/path/to/file', gs.best_params_['n_components'], gs.best_params_['C'])
    pca_logreg.run()

## Lessons Learned and Best Practices

- Follow the sci-kit learn API: https://scikit-learn.org/stable/developers/develop.html#rolling-your-own-estimator
- Favor array-like interface data structures internally in estimators. These work better across numpy, Dask, and Rapids that dataframe collections. If you need dataframe operations (like groupby, etc.) consider moving those to a transformer and converting to an array-like for processing.
- All attributes learned during .fit should be concrete, i.e. they should not be dask collections.
- To the extent possible, transformers should support
        numpy.ndarray
        pandas.DataFrame
        dask.Array
        dask.DataFrame
- If possible, transformers should accept a columns keyword to limit the transformation to just those columns, while passing through other columns untouched. inverse_transform should behave similarly (ignoring other columns) so that inverse_transform(transform(X)) equals X.
- Methods returning arrays (like .transform, .predict), should return the same type as the input. So if a dask.array is passed in, a dask.array with the same chunks should be returned.


last 4 from: https://ml.dask.org/contributing.html#conventions

## Using `dask-ml` to Train a Model

In [ ]:
from sklearn.datasets import make_classification
import pandas as pd

X_array, y_array = make_classification(
    n_samples=10_000,
    n_features=50,
    weights=[0.75, 0.25],
    flip_y=0.75,
    random_state=123,
)

X = pd.DataFrame(X_array, columns = [f"var{i}" for i in range(0,50)])
y = pd.Series(y_array)

In [ ]:
from dask_ml.datasets import make_classification_df

X_dask, y_dask = make_classification_df(
    n_samples=10_000,
    n_features=50,
#     weights=[0.75, 0.25],
#     flip_y=0.75,
    random_state=123,
    chunks=1000,
)

### Transformer: Add Feature

In [ ]:
import numpy as np
new_feature = pd.Series(np.random.randn(10_000))

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from abc import ABCMeta

class AddFeature(TransformerMixin, BaseEstimator, metaclass=ABCMeta):
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        """Add Feature to X"""
        # your code here
        
        return X

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from abc import ABCMeta
import dask.dataframe as dd

class AddFeature(TransformerMixin, BaseEstimator, metaclass=ABCMeta):
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        """Add Feature to X"""
        if isinstance(X, pd.DataFrame):
            X['var50'] = new_feature
        elif isinstance(X, dd.DataFrame):
            X['var50'] = dd.from_pandas(new_feature, npartitions=X.npartitions)
        
        return X

In [ ]:
AF = AddFeature()
X = AF.transform(X)
X

In [ ]:
AF = AddFeature()
X_dask = AF.transform(X_dask)
X_dask

### Custom Estimator

In [ ]:
from sklearn.base import BaseEstimator
import logging

logger = logging.getLogger('exp a')

class CustomEstimator(BaseEstimator):    
    def __init__(self, estimator, logger):
        self.estimator = estimator        
        self.logger = logger
    
    def fit(self, X, y=None, **fit_kws):
        self.estimator.fit(X, y)        
        self.logger.info("... log things ...")
        return self

In [ ]:
import xgboost as xgb

clf = CustomEstimator(xgb.XGBClassifier(n_jobs=-1, eval_metric='error', use_label_encoder=False), logger)
clf.fit(X, y)

In [ ]:
clf.estimator.feature_importances_

### Custom Estimator with Dask

In [ ]:
from dask.distributed import Client, progress

client = Client()
client

In [ ]:
# fit your custom estimator with Dask

In [ ]:
import xgboost.dask as dxgb

clf = CustomEstimator(dxgb.DaskXGBClassifier(n_jobs=-1, eval_metric='error', use_label_encoder=False), logger)
clf.fit(X_dask, y_dask)

### Pipeline

In [ ]:
# Remove the column we added from the previous transformation
X = X.drop(columns=['var50'])
X_dask = X_dask.drop(columns=['var50'])

In [ ]:
# confirm we've removed that column
X_dask

In [ ]:
# confirm we've removed that column
X

In [ ]:
from sklearn.pipeline import Pipeline

# your pipeline here

In [ ]:
from sklearn.pipeline import Pipeline

custom_estimator = CustomEstimator(xgb.XGBClassifier(n_jobs=-1, eval_metric='error', use_label_encoder=False), logger)
pipe = Pipeline(steps=[('add_feature', AddFeature()), ('custom_estimator', custom_estimator)])

pipe.fit(X, y)

In [ ]:
from sklearn.pipeline import Pipeline
import xgboost.dask as dxgb

custom_estimator_dask = CustomEstimator(dxgb.DaskXGBClassifier(n_jobs=-1, eval_metric='error', use_label_encoder=False), logger)
pipe = Pipeline(steps=[('add_feature', AddFeature()), ('custom_estimator_dask', custom_estimator_dask)])

pipe.fit(X_dask, y_dask)

In [ ]:
# confirm we added the feature
X

In [ ]:
# confirm we added the feature
X_dask

In [ ]:
custom_estimator.estimator.feature_importances_

In [ ]:
custom_estimator_dask.estimator.feature_importances_